---
title: 《Effective Python》 读书笔记
date: 
tags: ['Python']
mathjax: true

---
《Effective Python 59》 这本书可以说是目前为止我看到的对Python程序员最友好的一本进阶书籍。相比其他大而全的Python书籍，这本书有以下几个特点：  
<!-- more --> 
1. 从工程实践出发、以场景为主导进行阐述，配套的代码范例.
2. 内容不拖沓，没有基础的语法，基本都是进阶内容.
3. 不厚，只有二百多页（这一点至关重要）.  

这是我看完之后做的一份读书笔记，列出来以供以后不时查阅。



# 用Pythonic方式来思考

## bytes、str与unicode的区别
很多时候项目从Python2.x迁移到Python3.x会遇到字符编码的问题，原因是Pyhton2.x和Python3.x的字符编码不统一，具体如下：  
Python3：有两种表示字符序列的类型：`bytes`和`str`：
- `bytes`：8个二进制位
- `str`：`unicode`字符  

Python2：有两种表示字符序列的类型：`str`和`unicode`：
- `str`：8个二进制位
- `unicode`：`unicode`字符

`unicode`$\rightarrow$ 二进制: 常见的编码方式是`utf-8`，使用`encode`方法  

二进制$\rightarrow$`unicode`: 使用`decode`方法

在编程的时候，编解码放在接口外面来做。程序核心使用unicode字符，且不要对字符编码做任何假设。

In [1]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decoce('utf-8')
    else:
        value = bytes_or_str
    return value

def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value

> 在Python3中，涉及到文件处理的操作（使用内置的open函数）会默认的以UTF-8进行编码。而在Python2中默认采用二进制形式来编码。这也是导致很多意外事故发生的根源，特别是对于那些更习惯使用Python2的程序员而言。

比方说，将几个随机的二进制数据写入到一个文件中。在Python2中，下面的这段代码可以正常的工作，但是在Python3中却会报错并退出。

In [4]:
import os
with open('random.bin', 'w') as f:
    f.write(os.urandom(10))

TypeError: write() argument must be str, not bytes

导致这个异常发生的原因是在Python3中对于open函数又新增了一个名为encoding的参数。此参数默认为UTF-8。这样在文件句柄上进行read和write操作时，必须传入Unicode字符串的str实例，而不是包含了二进制数据的bytes实例。

## 用生成器表达式来改写数据量较大的列表推导  

下面这段代码只适合处理少量的输入值，对于大量数据，最好考虑生成器表达式而不是列表生成式

In [13]:
value = [len(x) for x in open('my_file.txt')]
value

[2, 3, 4, 5, 6, 7, 8, 9, 10, 3, 5, 7, 5, 5, 8, 11, 9, 3, 2, 2, 5, 5, 2]

- **使用圆括号构成生成器表达式**

In [14]:
value = (len(x) for x in open('my_file.txt'))
value

<generator object <genexpr> at 0x0000027C129F78E0>

In [5]:
next(value)

2

- 使用生成器表达式的另一个好处是可以互相组合

In [10]:
roots = ((x, x**10) for x in value)
roots

<generator object <genexpr> at 0x0000027C129F7CA8>

In [12]:
next(roots)

(3, 59049)

上面这种连锁生成器表达式，可以迅速在python中执行。如果要把多种手法组合起来，以操作大批量数据，最好是用生成器表达式实现

## 尽量用enumerate代替range

In [9]:
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for i in range(len(flavor_list)):
    print('%d: %s' % (i+1, flavor_list[i]))

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [10]:
for i, flavor in enumerate(flavor_list):
    print('%d: %s' % (i+1, flavor))

1: vanilla
2: chocolate
3: pecan
4: strawberry


- **还可以直接指定enumerate函数开始计数时所用的值**

In [11]:
for i, flavor in enumerate(flavor_list, 1):
    print('%d: %s' % (i, flavor))

1: vanilla
2: chocolate
3: pecan
4: strawberry


## 用zip函数同时遍历两个迭代器

In [15]:
names = ['LiMing', 'LiLi', 'ZhangMei']
letters = [len(n) for n in names]
longest_name = None
max_letters = 0

for i in range(len(names)):
    count = letters[i]
    if count > max_letters:
        max_letters = count
        longest_name = names[i]
    
        
print(longest_name, max_letters)

ZhangMei 8


- Python3中内置的zip函数，可以把两个或两个以上的迭代器封装为生成器，以便稍后求值，这种zip生成器，会从每个迭代器中获取该迭代器的下一个值，然后把这些值汇聚成元祖.  
- Python2中zip并不是生成器，对于大量数据，可能产生大量内存消耗.  

In [13]:
longest_name = None
max_letters = 0
for name, count in zip(names, letters):
    if count > max_letters:
        longest_name = name
        max_letters = count
        
print(longest_name, max_letters)

ZhangMei 8


## 不要在for和while循环后面写else块

## 合理利用 try/except/else/finally 结构中的每个代码块

- try...finally...  
这种结构简单的说是在`try`下的全部操作如果某项失败的话就终止并执行`finally`下定义的语句。如果全部操作都没有报错，那么最后也执行`finally`下定义的语句，经常用于既要向上传播异常，又要在异常发生时执行某些清理操作

In [16]:
handle = open('my_file.txt', 'w')
try:
    handle.write('111')
    handle.write('222')
finally:
    print('close handle')
    handle.close()

close handle


- try...except...else...  

In [17]:
import json
def load_json_key(data, key):
    try:
        result_dict = json.load_json_key(data)
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key]

- 混合使用

In [18]:
UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+')
    try:
        data = handle.read()
        op = json.loads(data)
        value = (op['numerator'], op['denominator'])
    except ZeroDivisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handle.write(result)
        return value
    finally:
        handle.close()

# 函数

## 尽量用异常来表示特殊情况，而不要返回None 

用`None`表示特殊情况如下：

In [20]:
def divide(a, b):
    try:
        return True, a/b
    except ZeroDivisionError:
        return False, None
    
sucess, result = divide(3,3)
result

1.0

用异常表示特殊情况如下：

In [18]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

try:
    result = divide(3, 0)
except ValueError:
    print('Invlid inputs')
else:
    print('Result is: %f' % result)

Invlid inputs


第一种方法问题在于，Python程序员习惯用`-`表示用不到的变量，那很有可能调用者会轻易的跳过元组的第一部。第二种方法让程序员不得不处理异常情况。

## 了解如何在闭包里使用外围作用域中的变量

假如有一份数字列表，要对其排序，但在排序时，要把出现在某个组群中的数字，放在组群外的那些数字之前，简单的实现如下：

In [21]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

numbers = [8, 3, 1, 2, 5, 4, 7, 6, -1]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
numbers

[2, 3, 5, 7, -1, 1, 4, 6, 8]

上面程序能正常工作的原因是以下三个方面：  
- Python支持闭包：闭包是一种定义在某个作用域中的函数，这种函数引用了那个作用域中的变量  
- Python的函数是一级对象，也就是说，我们可以直接引用函数、把函数赋给变量、把函数当成参数  
- Python使用特殊规则比较两个元祖，首先比较下标为0的对应元素，如果相等，再比较下标为1的对应元素，以此类推

若果增加一个功能，如果在数字出现在了组群中，返回一个标志，先试试下面这种写法：

In [22]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print(found)
print(numbers)

False
[2, 3, 5, 7, -1, 1, 4, 6, 8]


排序的结果是对的，但标志不对。解释如下：

当在表达式中引用变量的时候，Python解释器会按如下顺序遍历各作用域： 

- 当前函数的作用域。
- 任何外围作用域（比如其他的包含着的函数）。
- 包含当前代码的模块域（也称之为全局作用域）。
- 内置域（包含了像len,str等函数的域）。

如果上述地方都没找到，就抛出异常。

当给变量赋值时，如果变量在当前作用域内已经被定义过，那么该变量会具备新值，如果当前作用域没有这个变量，Python会把这次的赋值行为视为对变量的定义

在Python3中有一种特殊的写法，能够获取闭包内的数据，我们可以用`nonlocal`语句表明这样的意图。

In [24]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print(found)
print(numbers)

True
[2, 3, 5, 7, -1, 1, 4, 6, 8]


- `nonlocal`清楚的表明：如果在闭包内给该变量赋值，那么修改的其实是闭包外的那个作用域中的变量，`nonlocal`不能延伸到模块级别
- Python2中不支持`nonlocal`
- `nonlocal`可能也会像全局变量一样遭到滥用，建议只在及其简单的函数中使用这种机制
- 如果使用`nonlocal`的那些代码，已经写的越来越复杂了，那就应该将相关的状态封装成辅助类

下面定义的类与`nonlocal`的功能相同

In [21]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

numbers = [8, 3, 1, 2, 5, 4, 7, 6, -1]
group = {2, 3, 5, 7}
sorter = Sorter(group)
numbers.sort(key=sorter)
print(sorter.found)
print(numbers)

True
[2, 3, 5, 7, -1, 1, 4, 6, 8]


## 考虑用生成器来改写直接返回列表的函数

如果函数要产生一系列结果，最简单的做法是返回一份列表。例如，想知道一个字符串中每个单词的首字母在句子中的位置。代码如下：

In [25]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

address = 'Four score and seven years ago...'
result = index_words(address)
result

[0, 5, 11, 15, 21, 27]

这段代码的问题如下：

- 代码杂乱拥挤。每次都要调用`append`方法，而且要初始化`result`，返回`result`。
- 在返回前，要把所有的结果放在列表中，在输入量大的情况下，有内存崩溃的风险

下面这个生成器函数，返回一个迭代器，与之前的代码功能相同。

In [23]:
def index_words(text):
    if text:
        yield 0
    
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index+1
address = 'Four score and seven years ago...'
result = index_words(address)
result

<generator object index_words at 0x00000241086A9360>

In [24]:
list(result)

[0, 5, 11, 15, 21, 27]

## 在参数上迭代时，要多加小心

当一个函数接收的参数是一个对象列表，那么很有可能要在这个列表上迭代。如下代码计算个体占总体的百分比。

In [1]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for num in numbers:
        result.append(100 * num / total)
    return result

visits = [15, 35, 80]
normalize(visits)

[11.538461538461538, 26.923076923076923, 61.53846153846154]

如果将个体数据放在一份文件里，然后从文件中读取，我们定义`read_vists`函数，然后定义一个生成器以便将数据应用到更大数据集上。

In [5]:
def read_vists(path):
    with open(path) as f:
        for line in f:
            yield int(line)

it = read_vists('data.txt')
percentages = normalize(it)
percentages

[]

奇怪的是，以生成器返回的迭代器为参数，来调用`normalize`，没有产生任何结果。原因是迭代器只能产生一轮结果，在`sum`函数那里已经用完了。但之后在已经用完的迭代器上继续迭代时，没有报错。  
一种解决方法是通过参数来接受另外一个函数，那个函数每次调用后，都能返回新的迭代器。代码如下。

In [8]:
def normalize_func(numbers):
    total = sum(get_iter()) # 一个新的迭代器
    result = []
    for value in get_iter(): # 又一个新的迭代器
        percent = 100 * value / total
        result.append(percent)
    return result

这种方法显得生硬，很不Pythnic。另一种解决方法是新编一种实现迭代器协议的容器类。

> 实际上，当Python执行类似`for x in foo`这样的表达式的时候，它就会调用`iter(foo)`。内置的`iter`函数然后会调用`foo.__iter__`方法。该方法返回一个迭代器对象，而那个迭代器对象，则实现了`__next__`方法。然后循环语句会在迭代器对象上反复调用`next`方法，直到产生StopIteration异常。

只需自己的类把`__iter__`方法实现为生成器就满足上述要求。

In [11]:
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)
                
visits = ReadVisits('data.txt')
normalize(visits)

[7.6923076923076925,
 28.205128205128204,
 33.97435897435897,
 9.615384615384615,
 20.512820512820515]

`normalize`的`sum`方法会调用`ReadVisits.__iter__`方法，之后的`for`循环也会调用`ReadVisits.__iter__`方法。  

> 迭代器协议有这样的规定：如果把迭代器对象传给内置的`iter`函数，那么此函数会把该迭代器返回，反之，如果传给`iter`的是个容器类型的对象，那么`iter`函数则每次回返回新的迭代器对象，我们可以根据这种行为来判断输入值是不是迭代器对象本身，如果是，就抛出错误

下面是对第一个函数的完善。

In [29]:
def normalize_defensive(numbers):
    if iter(numbers) == iter(numbers):
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [30]:
visits = [15, 35, 80]
normalize_defensive(visits)

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [31]:
visits = ReadVisits('data.txt')
normalize_defensive(visits)

[7.6923076923076925,
 28.205128205128204,
 33.97435897435897,
 9.615384615384615,
 20.512820512820515]

In [32]:
it = read_vists('data.txt')
normalize_defensive(it)

TypeError: Must supply a container

## 用数量可变的位置参数减少视觉杂讯

In [33]:
def log(message, values):
    if not values:
        print(message)
    else:
        value_str = ','.join(str(x) for x in values)
        print('%s: %s' % (message, value_str))
log('My Number is: ', [11, 22, 33])

My Number is: : 11,22,33


In [34]:
def log(message, *values):
    if not values:
        print(message)
    else:
        value_str = ','.join(str(x) for x in values)
        print('%s: %s' % (message, value_str))
log('My Number is', 12, 33 ,44)

My Number is: 12,33,44


In [35]:
numbers = [12, 33, 44]
log('My Numver is', *numbers)

My Numver is: 12,33,44


## 十二. 用关键字参数表达可选的行为

## 十三. 用None和文档字符串来描述具有动态默认值的参数

例如打印日志时想要加上打印的时间。  
先看如下的写法：

In [36]:
from datetime import datetime
import time
def log(message, when=datetime.now()):
    print('%s: %s' % (when, message))

In [37]:
log('Hi there!')
time.sleep(1)
log('Hi again!')

2018-04-16 23:44:30.426229: Hi there!
2018-04-16 23:44:30.426229: Hi again!


从上面打印的信息来看，时间戳是相同的，这是因为`datetime.now`仅仅被执行了一次，而且是在函数被定义的时候。

在Python中，如果实现动态默认值，常用的方式是提供一个值为None的默认值，并且在帮助文档中记录详细的行为和使用方法。当代码发现一个值为None的参数的时候，就可以为其分配默认值了。

In [38]:
def log(message, when=None):
    """
    Log a message with a timestamp.

    Args:
        message: Message to print
        when: datetime of when the message occurred.
            Default to the present time.
    """
    when = datetime.now() if when is None else when
    print("%s: %s" %(when, message))

log("hi there!")
time.sleep(1)
log('hi again!')

2018-04-16 23:44:31.465338: hi there!
2018-04-16 23:44:32.466058: hi again!


In [39]:
import json
def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [40]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print(foo)
print(bar)

{'stuff': 5, 'meep': 1}
{'stuff': 5, 'meep': 1}


由于default参数的默认值只在模块加载时执行一次，所以凡是以默认的空字典调用这个函数的代码，都将共享一份字典

In [41]:
assert foo is bar

解决办法就是对关键字参数设置默认值None并且记录在该函数的说明文档中。

In [42]:
def decode(data, default=None):
    """Load JSON data from string.

    Args:
        data: JSON data to be decoded.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """

    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default
    
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}


## 十四. 用只能以关键字形式指定的参数来确保代码明确清晰

In [43]:
def safe_division_c(number, division, *, ignore_overflow=False, 
                    ignore_zero_division=False):
    pass

这些参数只能通过关键字参数赋值的形式被使用，而不是通过位置参数赋值的方式。

## 十五. 用@classmethod形式的多态去通用地构建对象

参考链接：  
https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner  
首先我们有一个处理时间的类：

In [44]:
class Date(object):
    
    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

如果我们要通过字符串创建许多的`Date`实例，此时我们要做如下操作：  
- 将string的日志转为int
- 通过int的日期构建`Date` 

如下：  
day, month, year = map(int, string_date.split('-'))  
date1 = Date(day, month, year)

对于这种情况，`C++`有重载的方法，但python没有重载，每个类只有一个构造器，只有一个`__init__`方法，因此`@classmethod`方法应运而生

In [45]:
class Date(object):

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1

`@staticmethod`与`classmethod`的区别是不构建类的实例，也不访问类，只是一个函数

In [46]:
class Date(object):

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1

    @staticmethod
    def is_date_valid(date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        return day <= 31 and month <= 12 and year <= 3999

date2 = Date.from_string('11-09-2012')
is_date = Date.is_date_valid('11-09-2012')